In [28]:
using LinearAlgebra

In [29]:
num_equations = 2
num_unknowns = 2

2

In [30]:
A = rand(num_equations, num_unknowns)

2×2 Matrix{Float64}:
 0.356659  0.88258
 0.608567  0.85262

In [31]:
b = rand(num_equations)

2-element Vector{Float64}:
 0.6223774196076156
 0.40438579374480343

In [32]:
sign = :<=

:<=

In [33]:
println("Original Inequalities:")
println()
for i = 1:size(A, 1)
    for j = 1:size(A, 2)
        print(A[i, j], "x", j)
        if(j != size(A, 1))
            print(" + ")
        end
    end
    println(" ", sense, " ", b[i])
    println()
end

Original Inequalities:

0.3566591842194946x1 + 0.8825797145787391x2 <= 0.6223774196076156

0.6085674664928984x1 + 0.8526202545582702x2 <= 0.40438579374480343



For the form of Ax <= b, and Ax >= b (ONLY)

Implementation assumes A has full row rank

Matrix A must have at least as many rows as columns (cannot be a underdetermined system)

In [34]:
function fourier_motzkin_elimination(A::Matrix{Float64}, b::Vector{Float64}, sign::Symbol)
    m, n = size(A)
    #Checking to make sure input matrix A size is correct
    if m < n
        throw(ArgumentError("Input matrix A must have at least as many rows as columns."))
    end
    
    #Used for swapping between <= and >=
    if sign == :<=
        A = -A
        b = -b
    elseif sign != :>=
        throw(ArgumentError("Invalid sign argument."))
    end
        
    for i = 1:n
        for j = i+1:n
            if A[i,i] == 0 && A[j,i] != 0
                # Swap rows to ensure A[i,i] != 0
                A[i,:], A[j,:] = A[j,:], A[i,:]
                b[i], b[j] = b[j], b[i]
            end
            if A[j,i] != 0
                # Eliminate variable xi from the jth inequality
                alpha = A[i,i] / A[j,i]
                A[j,:] = alpha * A[j,:] - A[i,:]
                b[j] = alpha * b[j] - b[i]
            end
        end
    end
            
    if sign == :<=
        A = -A
        b = -b
    end
            
    # Remove rows where all coefficients are zero
    nonzero_rows = [i for i in 1:size(A,1) if any(A[i,:] .!= 0)]
    A = A[nonzero_rows,:]
    b = b[nonzero_rows]
    
    #A are the coefficients
    #b are the constants
    return A, b
end


fourier_motzkin_elimination (generic function with 1 method)

In [35]:
coeffs, cons = fourier_motzkin_elimination(A, b, sense)

([0.3566591842194946 0.8825797145787391; -0.0 -0.38289009726772383], [0.6223774196076156, -0.385381662772071])

In [36]:
println("Updated inequalities:")
println()
for i = 1:size(coeffs, 1)
    for j = 1:size(coeffs, 2)
        print(coeffs[i, j], "x", j)
        if(j != size(coeffs, 1))
            print(" + ")
        end
    end
    println(" ", sign, " ", cons[i])
    println()
end

Updated inequalities:

0.3566591842194946x1 + 0.8825797145787391x2 <= 0.6223774196076156

-0.0x1 + -0.38289009726772383x2 <= -0.385381662772071

